結果の書き出し

In [21]:
import pandas as pd 
import os 
import pandas as pd 
import numpy as np 
import os
import matplotlib.pyplot as plt 
import networkx as nx
from datetime import datetime
from scipy.optimize import minimize
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import MultipleLocator
from matplotlib.lines import Line2D
from matplotlib.patches import Patch

base_path = '/Users/takahiromatsunaga/res2023'
df_link = pd.read_csv(os.path.join(base_path, 'results/stanw_link_post_all.csv')) #pd.DataFrame(data_link)
df_node = pd.read_csv(os.path.join(base_path, 'results/stanw_node_all.csv')) # pd.DataFrame(data_node)
df_res = pd.read_csv(os.path.join(base_path, 'results/vizu_20230130_1745_3.csv'))
np.set_printoptions(threshold=np.inf)


電車来たタイミングを再現するので，全部で15分間=50timestepでリンク交通量を加算する

In [30]:
num_res = df_res.values
print(num_res)

[[  0.   0.   0.   0.   0.   0.   0.   0.  42.  48.  42.  89. 111.  73.
   60.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  1.   1.   0.   0.   0.   0.   0.   0.  22.  41.  70.  88.  76.  89.
   30.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   3.   0.   0.   0.  18.   0.   6.   0.   0.   0.   0.
    0.  20.   0.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.]
 [  2.   2.   0.   0.   0.   0.   0.   0.  15.  43.  60.  82.  77.  65.
   29.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   4.   3.   0.   0.  12.  18.  17.   6.   0.   0.   0.
    0.  11.  20.   1.   2.   0.   0.  

In [32]:
num_res2 = num_res[:-1, 2:]
print(num_res2)

[[  0.   0.   0.   0.   0.   0.  42.  48.  42.  89. 111.  73.  60.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  22.  41.  70.  88.  76.  89.  30.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   3.   0.   0.   0.  18.   0.   6.   0.   0.   0.   0.   0.  20.
    0.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  15.  43.  60.  82.  77.  65.  29.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   4.   3.   0.   0.  12.  18.  17.   6.   0.   0.   0.   0.  11.
   20.   1.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

In [39]:
## 加算用の親行列（15分間用）
all_num = np.zeros((80, 70))

for i in range(14):
    all_num[i*5:i*5+15, :] += num_res2
#print(num_res2)

#all_num[35:50, :] += num_res2

print(all_num)

[[  0.   0.   0.   0.   0.   0.  42.  48.  42.  89. 111.  73.  60.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  22.  41.  70.  88.  76.  89.  30.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   3.   0.   0.   0.  18.   0.   6.   0.   0.   0.   0.   0.  20.
    0.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.  15.  43.  60.  82.  77.  65.  29.   0.
    0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   4.   3.   0.   0.  12.  18.  17.   6.   0.   0.   0.   0.  11.
   20.   1.   2.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

In [40]:
num_15min = all_num[15:65, :]
print(num_15min.shape)

(50, 70)


In [41]:
# 足されたやつをdataframeにする→グラフ上での可視化のため
res_15min = pd.DataFrame(num_15min, columns=[i for i in range(1, 71)])

In [42]:
print(res_15min)

     1    2     3    4    5    6     7     8      9      10  ...   61   62  \
0   1.0  1.0  13.0  1.0  0.0  0.0  49.0  82.0  123.0  144.0  ...  0.0  2.0   
1   1.0  2.0  10.0  3.0  0.0  0.0  27.0  65.0  117.0  140.0  ...  0.0  0.0   
2   0.0  2.0   3.0  1.0  0.0  0.0  19.0  64.0   95.0  144.0  ...  0.0  0.0   
3   1.0  1.0   8.0  2.0  0.0  0.0  13.0  63.0   87.0  110.0  ...  0.0  0.0   
4   2.0  2.0   2.0  7.0  0.0  0.0  10.0  41.0   64.0  124.0  ...  0.0  0.0   
5   1.0  1.0  13.0  1.0  0.0  0.0  49.0  82.0  123.0  144.0  ...  0.0  2.0   
6   1.0  2.0  10.0  3.0  0.0  0.0  27.0  65.0  117.0  140.0  ...  0.0  0.0   
7   0.0  2.0   3.0  1.0  0.0  0.0  19.0  64.0   95.0  144.0  ...  0.0  0.0   
8   1.0  1.0   8.0  2.0  0.0  0.0  13.0  63.0   87.0  110.0  ...  0.0  0.0   
9   2.0  2.0   2.0  7.0  0.0  0.0  10.0  41.0   64.0  124.0  ...  0.0  0.0   
10  1.0  1.0  13.0  1.0  0.0  0.0  49.0  82.0  123.0  144.0  ...  0.0  2.0   
11  1.0  2.0  10.0  3.0  0.0  0.0  27.0  65.0  117.0  140.0  ...